# Telco Customer Churn - EDA & Baseline Modeling


## Business Problem

Customer churn is costly: acquiring new customers often is more costly then retaining the existing ones. The goal of this project is to create a model that will **predict weather a Customer will churn or not** and to identify which factors are most associated with churn, so a business can prioritize retention actions.

## Objective
- Build ML model that predicts churn (binary classification).
- Evaluate model preformance using appropriate classification metrics (precision, recall, F1)
- Produce interpretable insights

## Approach

1. Load and validate data quality
2. clean and prepare features
3. EDA to understand churn patterns
4. Build a baseline ML model
5. Summarize Insights

## Dataset
Telco Customer Churn dataset from Kaggle. Each row represents a customer. Each column represents a customers attributes.

## 1. Imports

In [76]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

## 2. Loading Data

We begin by loading the dataset and checking its basic structure.

In [77]:
dataPath = "../data/raw/archive/WA_Fn-UseC_-Telco-Customer-Churn.csv"

df = pd.read_csv(dataPath)

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [78]:
print("Shape:", df.shape)
print("\nColumns:\n", df.columns)

Shape: (7043, 21)

Columns:
 Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')


**Result:** The dataset contains 7043 customers and 21 columns, including demographics, service subscriptions, billing information, and the target variable churn.

## 3.  Data Quality Checks & Cleaning

- Column data types (numeric vs categorical)
- Missing values
- Potential type issues

This is required since ML models need consistent numeric formats.

In [79]:
print("\n--- INFO ---")
df.info()



--- INFO ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-n

In [80]:
df.dtypes.value_counts()

object     18
int64       2
float64     1
Name: count, dtype: int64

In [81]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

df.isna().sum().sort_values(ascending=False).head()


TotalCharges     11
gender            0
SeniorCitizen     0
Partner           0
customerID        0
dtype: int64

In [82]:
df = df.dropna(subset=["TotalCharges"])

### 3.1 Fix TotalCharges Type

In this dataset, TotalCharges sometimes appears as a string (object dtype).
Some rows contain blanks which are not counted as NaN.
We convert Total Charges to numeric with 'errors="coerce"' to turn invalid values to NaN and then handle those missing rows.

**Decision:** Drop small number of rows with missing TotalCharges after conversion (11 rows)

## 4. Exploratory Data Analysis

We explore how churn differs across key customer attributes using:

- Summary Tables (Value counts, Crosstabs, pivot tables)
- Aggregations (groupby + agg)
- Visualizations (distributions and comparisons)


In [83]:
df["Churn"].value_counts()

Churn
No     5163
Yes    1869
Name: count, dtype: int64

In [84]:
(df["Churn"]
 .value_counts(normalize=True)  # proportions that sum to 1
 .mul(100)                      # convert to %
 .round(2))                     # rounding

Churn
No     73.42
Yes    26.58
Name: proportion, dtype: float64

### 4.1 Numeric Feature Summary

'describe()' summarizes numeric columns (count/mean/std/min/max and quartiles)

In [85]:
df.describe().round(2)

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7032.00,7032.00,7032.00,7032.00
mean,0.16,32.42,64.80,2283.30
std,0.37,24.55,30.09,2266.77
min,0.00,1.00,18.25,18.80
25%,0.00,9.00,35.59,401.45
50%,0.00,29.00,70.35,1397.48
75%,0.00,55.00,89.86,3794.74
max,1.00,72.00,118.75,8684.80


### Numeric Summary

- **Tenure** ranges from new customers to long-long term customers, churn is often higher among low tenure customers, so tenure is likely an important attribute.
- **MonthlyCharges** shows meaningful spread, suggested customers likely have very differenc plans.
- **TotalCharges** is cumulative billing and tends to increase with tenure; low totals often correspond with customers with very short tenure.

In [86]:
pd.crosstab(df["Contract"], df["Churn"], normalize="index").sort_values("Yes", ascending=False)

Churn,No,Yes
Contract,,
Month-to-month,0.572903,0.427097
One year,0.887228,0.112772
Two year,0.971513,0.028487


### Churn rate by contract

- Month-to-month customers churn at a much higher rate than customers on 1-year or 2-year contract.
- Longer contracts are associated with significantly lower churn, suggesting **contract type** is a strong churn driver.
- This points to targeting and converting Month-to-month customers with incentives to move to longer-term plans.

### 4.2 Churn rates for other categorical features

After checking Contract, we examine other categorical features to see which groups have higher churn risk.
Instead of raw counts, we compute **churn rate** (The proportion of customers with 'Churn = Yes')

In [88]:
def churn_rate(series):
    """
    Compute churn rate as the fraction of rows where churn == "Yes".
    Assumes series contains strings like "Yes"/"No".
    """
    return (series == "Yes").mean()

churn_rate(df["Churn"])


np.float64(0.26578498293515357)

In [94]:
categorical_to_check = ["InternetService", "PaymentMethod"]


for col in categorical_to_check:

    rates = (df.groupby(col))["Churn"].apply(churn_rate).sort_values(ascending=False)

    print(f"\nChurn rate by {col}:\n")
    display(rates.round(3))


Churn rate by InternetService:



InternetService
Fiber optic    0.419
DSL            0.190
No             0.074
Name: Churn, dtype: float64


Churn rate by PaymentMethod:



PaymentMethod
Electronic check             0.453
Mailed check                 0.192
Bank transfer (automatic)    0.167
Credit card (automatic)      0.153
Name: Churn, dtype: float64

**Key Observations:**

- **InternetService:** Customers with **Fiber optic** have the highest churn rate ~41.9%, followed by **DSL** ~19.0%. Customers with **no internet service** have the lowest churn rate ~7.4%
-**PaymentMethod:** **Electronic Check** has the highest churn rate ~45.3%, substantially higher than the other payment methods ~15-19%.

These gaps suggest InternetService and PaymentMethod are likely strong predictors of churn and represent actionable segments. Reduce reliance on electronic check by encouraging autopay.